In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.2 MB/s eta 0:00:00


In [2]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
from torch.utils.data import Dataset
from torchvision.datasets import CIFAR100
import torchvision.transforms as T


class CIFAR100Dataset(Dataset):
    """CIFAR100 dataset.

    Feature images are automatically flattened.

    Parameters
    ----------
    root : str
        Directory where the actual data is located (or downloaded to).

    train : bool
        If True the training set is returned (60_000 samples). Otherwise
        the validation set is returned (10_000 samples).

    Attributes
    ----------
    tv_dataset : CIFAR100
        Instance of the torchvision `CIFAR100` dataset class.
    """

    def __init__(self, root, train=True, download=True):
      # if train:
      transform= T.Compose([   
        T.Resize(256),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize([0.49139968, 0.48215841, 0.44653091], [0.24703223 , 0.24348513, 0.26158784])
      ]) 
      # else:
      #     transform= T.Compose([   
      #       T.Resize(256),
      #       T.ToTensor(),
      #     ]) 

      self.tv_dataset = CIFAR100(
            root, 
            train=train,
            download=download,
            transform=transform,
        )

    def __len__(self):
        """Get the length of the dataset."""
        return len(self.tv_dataset)

    def __getitem__(self, ix):
        """Get a selected sample.

        Parameters
        ----------
        ix : int
            Index of the sample to get.

        Returns
        -------
        x : torch.Tensor
            Flattened feature tensor of shape `(784,)`.

        y : torch.Tensor
            Scalar representing the ground truth label. Number between 0 and 9.
        """
        return self.tv_dataset[ix]


In [4]:
import math

import torch
import torch.nn as nn
from torch.nn.utils.prune import l1_unstructured, random_unstructured


class AlexNet(nn.Module):
    """AlexNet convolution network.

    Parameters
    ----------
    image_channels : int
        Number of channels in input picture (color pictures in CIFAR100 have 3 channels).

    out_features : int
        Number of target classes (100 for CIFAR100).

    Attributes
    ----------
    module_list : nn.ModuleList
        List holding all the layers in the right order.
    """

    def __init__(self, image_channels, out_features):
        super().__init__()

        # self.features= nn.Sequential(
        # nn.Conv2d(image_channels, 96, kernel_size=11, stride=4, padding=0),
        # nn.ReLU(inplace=True),
        # nn.MaxPool2d(kernel_size=3, stride=2),
        # #2
        # nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
        # nn.ReLU(inplace=True),
        # nn.MaxPool2d(kernel_size=3, stride=2),
        # #3
        # nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
        # nn.ReLU(inplace=True),
        # #4
        # nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
        # nn.ReLU(inplace=True),
        # #5
        # nn.Conv2d(384, 256, kernel_size=5, stride=1, padding=2),
        # nn.ReLU(inplace=True),
        # nn.MaxPool2d(kernel_size=3, stride=2),
        # )
        # self.classifier= nn.Sequential(
        #     nn.Dropout(), nn.Linear(256*6*6, 4096), #128*2*2, 1024
        # nn.ReLU(inplace=True), nn.Dropout(),
        # nn.Linear(4096, out_features=out_features)
        # )

        channel_sizes = (image_channels,) + (96, 256, 384, 384, 256,)
        conv_kernel_sizes = (11, 5, 3, 3, 5,)
        conv_strides = (4, 1, 1, 1, 1, )
        conv_paddings = (0, 2, 1, 1, 2)
        layer_list = []

        for i in range(len(channel_sizes) - 1):
            layer_list.append(nn.Conv2d(channel_sizes[i], channel_sizes[i + 1], 
                                        kernel_size=conv_kernel_sizes[i], 
                                        stride=conv_strides[i], 
                                        padding=conv_paddings[i]))

        layer_sizes = (256*6*6, 4096, out_features)
        for i in range(len(layer_sizes) - 1):
            layer_list.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))

        self.module_list = nn.ModuleList(layer_list)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)

    def forward(self, x):
        """Run the forward pass.

        Parameters
        ----------
        x : torch.Tensor
            Batch of features of shape `(batch_size, n_features)`.

        Returns
        -------
        torch.Tensor
            Batch of predictions (logits) of shape `(batch_size, n_targets)`.
        """
        x = self.maxpool(self.relu(self.module_list[0](x)))
        x = self.maxpool(self.relu(self.module_list[1](x)))
        x = self.relu(self.module_list[2](x))
        x = self.relu(self.module_list[3](x))
        x = self.maxpool(self.relu(self.module_list[4](x)))
        x = x.view(x.size(0), 256*6*6)
        x = self.dropout(x)
        x = self.relu(self.module_list[5](x))
        x = self.dropout(x)
        x = self.module_list[6](x)
        return x


def prune_layer(layer, prune_ratio=0.3, method="l1"):
    """Prune a linear layer.

    Modifies the module in-place. We make an assumption that the bias
    is included.

    Parameters
    ----------
    linear : nn.Linear
        Linear module containing a bias.

    prune_ratio : float
        Number between 0 and 1 representing the percentage of weights
        to prune.

    method : str, {"l1", "random"}
        Pruning method to use.
    """
    if method == "l1":
        prune_func = l1_unstructured
    elif method == "random":
        prune_func = random_unstructured
    else:
        raise ValueError

    prune_func(layer, "weight", prune_ratio)
    prune_func(layer, "bias", prune_ratio)

# тут все аналогичным образом поддерживается и для conv слоев, так 
# что отличия минимальны или их вообще не будет
def prune_net(net, prune_ratio=0.3, method="l1"):
    """Prune each layer of the multilayer perceptron.

    Modifies the module in-place. We make an assumption that each
    linear layer has the bias included.

    Parameters
    ----------
    net : net
        Multilayer perceptron instance.

    prune_ratio : float or list
        Number between 0 and 1 representing the percentage of weights
        to prune. If `list` then different ratio for each
        layer.

    method : str, {"l1", "random"}
        Pruning method to use.
    """
    if isinstance(prune_ratio, float):
        prune_ratios = [prune_ratio] * len(net.module_list)
    elif isinstance(prune_ratio, list):
        if len(prune_ratio) != len(net.module_list):
            raise ValueError("Incompatible number of prune ratios provided")

        prune_ratios = prune_ratio
    else:
        raise TypeError

    for prune_ratio, layer in zip(prune_ratios, net.module_list):
        prune_layer(layer, prune_ratio=prune_ratio, method=method)


def check_pruned_layer(layer):
    """Check if a Linear module was pruned.

    We require both the bias and the weight to be pruned.

    Parameters
    ----------
    linear : nn.Linear
        Linear module containing a bias.

    Returns
    -------
    bool
        True if the model has been pruned.
    """
    params = {param_name for param_name, _ in layer.named_parameters()}
    expected_params = {"weight_orig", "bias_orig"}

    return params == expected_params


# тут тоже все как раньше, потому что сверточный слой это тоже
# попросту тензор и мы можем его просто взять и скопировать
# так же, как и матрицу в линейных слоях раньше
def copy_weights_layer(layer_unpruned, layer_pruned):
    """Copy weights from an unpruned model to a pruned model.

    Modifies `layer_pruned` in place.

    Parameters
    ----------
    layer_unpruned : nn.Linear
        Linear model with a bias that was not pruned.

    layer_pruned : nn.Linear
        Linear model with a bias that was pruned.
    """
    assert check_pruned_layer(layer_pruned)
    assert not check_pruned_layer(layer_unpruned)

    with torch.no_grad():
        layer_pruned.weight_orig.copy_(layer_unpruned.weight)
        layer_pruned.bias_orig.copy_(layer_unpruned.bias)


def copy_weights_net(net_unpruned, net_pruned):
    """Copy weights of an unpruned network to a pruned network.

    Modifies `net_pruned` in place.

    Parameters
    ----------
    net_unpruned : AlexNet
        MLP model that was not pruned.

    net_pruned : MLP
        MLP model that was pruned.
    """
    zipped = zip(net_unpruned.module_list, net_pruned.module_list)

    for layer_unpruned, layer_pruned in zipped:
        copy_weights_layer(layer_unpruned, layer_pruned)


def compute_stats(net):
    """Compute important statistics related to pruning.

    Parameters
    ----------
    net : MLP
        Multilayer perceptron.

    Returns
    -------
    dict
        Statistics.
    """
    stats = {}
    total_params = 0
    total_pruned_params = 0

    for layer_ix, linear in enumerate(net.module_list):
        assert check_pruned_layer(linear)

        weight_mask = linear.weight_mask
        bias_mask = linear.bias_mask

        params = weight_mask.numel() + bias_mask.numel()
        pruned_params = (weight_mask == 0).sum() + (bias_mask == 0).sum()

        total_params += params
        total_pruned_params += pruned_params

        stats[f"layer{layer_ix}_total_params"] = params
        stats[f"layer{layer_ix}_pruned_params"] = pruned_params
        stats[f"layer{layer_ix}_actual_prune_ratio"] = pruned_params / params

    stats["total_params"] = total_params
    stats["total_pruned_params"] = total_pruned_params
    stats["actual_prune_ratio"] = total_pruned_params / total_params

    return stats


In [12]:
import torch
import torch.nn as nn
import tqdm
from torch.utils.data import DataLoader

import wandb
# from data import MNISTDataset
# from utils import MLP, compute_stats, copy_weights_net, prune_net, reinit_net


def loop_dataloader(dataloader):
    """Loop infinitely over a dataloader.

    Parameters
    ----------
    dataloader : DataLoader
        DataLoader streaming batches of samples.

    Yields
    ------
    X_batch : torch.Tensor
        Batch of features.

    y_batch : torch.Tensor
        Batch of predictions.
    """
    while True:
        for x in iter(dataloader):
            yield x


def train(model, dataloader_train, loss_inst, optimizer, max_iter=10_000,
          dataloader_val=None, val_freq=500):
    """Run the training loop.

    Parameters
    ----------
    model : nn.Module
        Neural network (in our case MLP).

    dataloader_train : DataLoader
        Dataloader yielding training samples.

    loss_inst : callable
        Computes the loss when called.

    optimizer : torch.optim.Optimizer
        Instance of an optimizer.

    max_iter : int
        The number of iterations we run the training for
        (= number of graident descent steps).

    dataloader_val : None or DataLoader
        Dataloader yielding validation samples. If provided it will
        also single to us that we want to track metrics.

    val_freq : int
        How often evaluation run.
    """
    global device
    iterable = loop_dataloader(dataloader_train)
    iterable = tqdm.tqdm(iterable, total=max_iter)
    it = 0
    for X_batch, y_batch in iterable:
        if it == max_iter:
            break
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logit_batch = model(X_batch)

        loss = loss_inst(logit_batch, y_batch)
        if dataloader_val is not None:
            wandb.log({"loss": loss}, step=it)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if it % val_freq == 0 and dataloader_val is not None:
            is_equal = []

            for X_batch_val, y_batch_val in dataloader_val:
                X_batch_val = X_batch_val.to(device)
                y_batch_val = y_batch_val.to(device)
                is_equal.append(
                    model(X_batch_val).argmax(dim=-1) == y_batch_val
                )

            is_equal_t = torch.cat(is_equal)
            acc = is_equal_t.sum() / len(is_equal_t)
            wandb.log({"accuracy_val": acc}, step=it)

        it += 1



def experiment(MAX_ITERS, PRUNE_ITERS, PRUNE_METHODS, PRUNE_RATIOS , RANDOM_STATES, project_name, start_from):
    args = {"max-iter" : 50000, "batch-size" : 64, "prune-iter" : 1,
          "prune-method" : "l1", "prune-ratio" : 0.2, "val-freq" : 250, "random-state" : 1}
    i = 0;
    for max_iter in MAX_ITERS:
      for prune_iter in PRUNE_ITERS:
        for prune_method in PRUNE_METHODS:
          for prune_ratio in PRUNE_RATIOS:
            for random_state in RANDOM_STATES:
              if i < start_from:
                i += 1
                continue
              args["max-iter"] = max_iter
              args["prune-iter"] = prune_iter
              args["prune-method"] = prune_method
              args["prune-ratio"] = prune_ratio
              args["random-state"] = random_state

              wandb.init(
                  project=project_name,
                  entity="bspanfilov",
                  config=args,
              )
              wandb.define_metric("accuracy_val", summary="max")

              dataset_train = CIFAR100Dataset(
                  "data",
                  train=True,
                  download=True,
              )
              dataset_val = CIFAR100Dataset(
                  "data",
                  train=False,
                  download=True,
              )

              if args["random-state"] is not None:
                  torch.manual_seed(args["random-state"])

              dataloader_train = DataLoader(
                  dataset_train, batch_size=args["batch-size"], shuffle=True
              )
              dataloader_val = DataLoader(
                  dataset_val, batch_size=args["batch-size"], shuffle=True
              )

              kwargs = dict(
                  image_channels=3,
                  out_features=100,
              )

              net = AlexNet(**kwargs).to(device)

              net_copy = AlexNet(**kwargs).to(device)
              net_copy.load_state_dict(net.state_dict())

              loss_inst = nn.CrossEntropyLoss()
              optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)

              # Train and prune loop
              if args["prune-ratio"] > 0:
                  per_round_prune_ratio = 1 - (1 - args["prune-ratio"]) ** (1 / args["prune-iter"])

                  per_round_prune_ratios = [per_round_prune_ratio] * len(net.module_list)
                  per_round_prune_ratios[-1] /= 2

                  per_round_max_iter = int(args["max-iter"] / args["prune-iter"])

                  for prune_it in range(args["prune-iter"]):
                      train(
                          net,
                          dataloader_train,
                          loss_inst,
                          optimizer,
                          max_iter=per_round_max_iter,
                      )
                      prune_net(net, per_round_prune_ratios, method=args["prune-method"])

                      copy_weights_net(net_copy, net)
                      # надо будет добавить эксперементов без копирования, вдруг тоже что-то получиться
                      # а пока что фигачу кучу моделей на mnist

                      stats = compute_stats(net)
                      for name, stat in stats.items():
                          summary_name = f"{name}_pruneiter={prune_it}"
                          wandb.run.summary[summary_name] = stat

              # Run actual training with a final pruned network
              train(
                  net,
                  dataloader_train,
                  loss_inst,
                  optimizer,
                  max_iter=args["max-iter"],
                  dataloader_val=dataloader_val,
                  val_freq=args["val-freq"],
              )




In [ ]:
# тут просто настройки, с которыми я как раз запускал эксперименты
MAX_ITERS=[11000]
PRUNE_ITERS=[5]
PRUNE_METHODS=['l1']
PRUNE_RATIOS=[0.8, 0.85]
RANDOM_STATES=[2]

experiment(MAX_ITERS, PRUNE_ITERS, PRUNE_METHODS, PRUNE_RATIOS, RANDOM_STATES, "thinking", 0)

accuracy_val,▇█▁▇▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
loss,▅█▂▄▇▂▃▅▅▅▅▃▄▅▄▄▁▄▃▄▂▄▂▃▄▅▇▄▂▄▁▃▄▃▂▂▃▅▂▅
loss,4.60824


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 11000/11000 [50:53<00:00,  3.60it/s]


accuracy_val,▁▂▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████▇███████████████
loss,█▇▆▆▅▅▄▅▅▄▄▄▄▄▃▄▃▄▃▃▃▃▃▂▄▃▁▃▂▃▃▂▂▃▂▂▂▁▂▂
loss,0.9699


Files already downloaded and verified
Files already downloaded and verified


...

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 11000/11000 [48:41<00:00,  3.77it/s]


accuracy_val,▁▂▂▃▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇███████████
loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▄▂▄▃▃▃▂▂▂▂▃▃▂▂▂▃▂▃▂▂▂▂▂▁▁▂
actual_prune_ratio_pruneiter=0,0.241
actual_prune_ratio_pruneiter=1,0.42378
actual_prune_ratio_pruneiter=2,0.56243
actual_prune_ratio_pruneiter=3,0.66761
actual_prune_ratio_pruneiter=4,0.74741
layer0_actual_prune_ratio_pruneiter=0,0.24213
layer0_actual_prune_ratio_pruneiter=1,0.42565
layer0_actual_prune_ratio_pruneiter=2,0.5647
layer0_actual_prune_ratio_pruneiter=3,0.6701


Files already downloaded and verified
Files already downloaded and verified


 23%|██▎       | 2480/11000 [11:38<24:29,  5.80it/s]